
# SPARK IN PYTHON 

Spark is a unified analytics engine for large scale data procession. They work 100x faster then other as an example Hadoop.

Can be written in Java, Scala, Python, R and SQL which is why it is called a `Unified analytics engine`.

Python and R is being translated to Spark session and to the executor. It is runed on Java and is therefore important to have it installed. 

For Pyspark before 3.2 we need to use `Koalas` with pyspark.pandas but today it is integrated.  

## Databricks

Started as a resarch project. Became a company that levarce Spark. 

## VS Code

We have a extension so we can use Databricks. 

Here we can use cluster and the different language. 

## INSTALLATION REQUIREMENTS 

1. INSTALL PYTHON
2. INSTALL JAVA
3. INSTALL PYSPARK==3.2.0 (THIS ALSO INSTALLS SPARK V3.2.0 IN YOUR ENV)


## LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

# USING PYSPARK==3.2 (BUILT IN PANDAS)
from pyspark import pandas as ps # Alow us to use pandas and pyspark.
from pyspark.sql import SparkSession # Give us a session


In [2]:
# SETTING UP SPARK SESSION ----

spark = (
    SparkSession.builder 
    .master("local[12]") # all cores
    .appName("lab_66_pyspark")  # name of app
    # Configuation
    .config("spark.driver.bindAddress","localhost") # ui part
    .config("spark.ui.port", "4050") 
    .config("spark.driver.memory", "16g") # memory so we dont clase
    .config("spark.memory.fraction", "0.9") 
    .getOrCreate() # 
)

spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/24 21:37:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Can look the UI path. 

Here we can see the jobs and the information. 

In [ ]:
spark.version

## SPARK WEB UI

Should be running on localhost:4050
Will use this to examine memory usage, jobs, and data cache

In [3]:
spark


## 1.0 PYSPARK PANDAS

Available in PySpark v3.2.0

In [6]:
# NASDAQ ANALYSIS ----
# - About the data: 
#   I pulled from R using my tidyquant package 
#   This was done in Lab 65: Spark in R
#   It contains 5.8M rows & 4000 time series groups

nasdaq_df = ps.read_csv(r'/Users/lucasbagge/Documents/Projects/business\ sciecne\ /lab_66_pyspark/01-data-raw/nasdaq_data.csv')
nasdaq_df.head()

23/01/24 21:43:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:43:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,symbol,date,adjusted
0,AACG,2011-01-03,0.297095
1,AACG,2011-01-04,0.300307
2,AACG,2011-01-05,0.297095
3,AACG,2011-01-06,0.308336
4,AACG,2011-01-07,0.309139


In [7]:
nasdaq_df.shape

(5795746, 3)

In [8]:
nasdaq_df['symbol'].value_counts().shape

(3980,)

In [9]:
nasdaq_df.head().info()

23/01/24 21:43:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   symbol    5 non-null      object
 1   date      5 non-null      object
 2   adjusted  5 non-null      object
dtypes: object(3)

In [10]:
# Change data types 
nasdaq_df['symbol'] = nasdaq_df['symbol'].astype("str")
nasdaq_df['date'] = nasdaq_df['date'].astype("datetime64")
nasdaq_df['adjusted'] = nasdaq_df['adjusted'].astype("float64")

In [11]:
# What is spark doing?
nasdaq_df.spark.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [(cast(_we0#73 as bigint) - 1) AS __index_level_0__#71L, CASE WHEN isnull(symbol#58) THEN None ELSE symbol#58 END AS symbol#275, cast(date#59 as timestamp) AS date#278, cast(adjusted#60 as double) AS adjusted#281]
   +- Window [row_number() windowspecdefinition(_w0#72L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS _we0#73], [_w0#72L ASC NULLS FIRST]
      +- Sort [_w0#72L ASC NULLS FIRST], false, 0
         +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#376]
            +- Project [symbol#58, date#59, adjusted#60, monotonically_increasing_id() AS _w0#72L]
               +- FileScan csv [symbol#58,date#59,adjusted#60] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/lucasbagge/Documents/Projects/business sciecne /lab_66_pys..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<symbol:string,date:string,adjusted:str

23/01/24 21:44:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:44:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [12]:
# CHECKPOINTING ----
# - Can help to speed up spark by Caching operations
# - IF YOU GET WARNING NOT ENOUGH MEMORY HERE: Adjust your .config("spark.driver.memory", "16g") 
spark 

nasdaq_df = nasdaq_df.spark.local_checkpoint()

23/01/24 21:44:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:44:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:44:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [13]:
# LAG OPERATION ----

nasdaq_shifted_df = nasdaq_df \
    .drop('date') \
    .groupby("symbol") \
    .shift(1) \
    .rename(columns={'adjusted':'lag1'}) \
    .sort_index()


In [14]:
# Create another local checkpoint to speed up 
nasdaq_shifted_df = nasdaq_shifted_df.spark.local_checkpoint()

nasdaq_shifted_df.head()

# COMBINE LAG WITH ADJUSTED PRICES ----
# - IF YOU GET ERROR: NEED TO SET OPTION TO COMPUTE ON MULTIPLE DATA FRAMES

ps.set_option('compute.ops_on_diff_frames', True)

nasdaq_lag_df = nasdaq_df.copy()
nasdaq_lag_df['lag1'] = nasdaq_shifted_df[['lag1']]

nasdaq_lag_df.head()

nasdaq_lag_df = nasdaq_lag_df.spark.local_checkpoint()

In [28]:
nasdaq_lag_df.head()

,symbol,date,adjusted,lag1
0,AACG,2011-01-03,0.297095,NaN
7,AACG,2011-01-12,0.309942,0.305125
19,AACG,2011-01-31,0.336440,0.324396
22,AACG,2011-02-03,0.335637,0.336440
26,AACG,2011-02-09,0.336440,0.325199


In [29]:
# COMBINE LAG WITH ADJUSTED PRICES ----
# - IF YOU GET ERROR: NEED TO SET OPTION TO COMPUTE ON MULTIPLE DATA FRAMES

ps.set_option('compute.ops_on_diff_frames', True)

nasdaq_lag_df = nasdaq_df.copy()
nasdaq_lag_df['lag1'] = nasdaq_shifted_df[['lag1']]

nasdaq_lag_df = nasdaq_lag_df.spark.local_checkpoint()
    
# SUMMARIZE RETURNS

nasdaq_agg_df = nasdaq_lag_df \
    .assign(returns = lambda x: (x['adjusted'] / x['lag1']) - 1) \
    .groupby('symbol') \
    .aggregate(
        {
            'returns': ['mean', 'std', 'count'],
            'date': ['max', 'min']
        }
    ) \
    .reset_index()

nasdaq_agg_df.columns = ["_".join(a) for a in nasdaq_agg_df.columns.to_flat_index()]

nasdaq_agg_df = nasdaq_agg_df.spark.local_checkpoint()

23/01/24 21:50:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:50:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:50:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Py4JJavaError: An error occurred while calling o5244.localCheckpoint.
: java.lang.NoSuchMethodError: 'org.apache.spark.internal.config.ConfigEntry org.apache.spark.sql.internal.SQLConf$.ENABLE_TWOLEVEL_AGG_MAP_PARTIAL_ONLY()'
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.checkIfFastHashMapSupported(HashAggregateExec.scala:674)
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.enableTwoLevelHashMap(HashAggregateExec.scala:681)
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.doProduceWithKeys(HashAggregateExec.scala:698)
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.doProduce(HashAggregateExec.scala:151)
	at org.apache.spark.sql.execution.CodegenSupport.$anonfun$produce$1(WholeStageCodegenExec.scala:96)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:222)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQu

In [26]:

# JOIN NASDAQ INDEX META DATA ----

nasdaq_index_df = ps.read_csv(r'/Users/lucasbagge/Documents/Projects/business\ sciecne\ /lab_66_pyspark/01-data-raw/nasdaq_index.csv')

nasdaq_index_df = nasdaq_index_df[['symbol', 'company', 'market.cap']]

nasdaq_index_df['market.cap'] = nasdaq_index_df['market.cap'].astype('float64')

nasdaq_index_df['symbol'] = nasdaq_index_df['symbol'].astype('str')

nasdaq_index_df['company'] = nasdaq_index_df['company'].astype('str')

# JOIN

nasdaq_agg_join_df = nasdaq_agg_df \
    .rename({'symbol_':'symbol'}, axis = 1) \
    .set_index('symbol') \
    .join(nasdaq_index_df.set_index('symbol'), how = "left", lsuffix = "_l", rsuffix = "_r") \
    .reset_index()

nasdaq_agg_join_df.head()

23/01/24 21:49:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:49:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:49:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:49:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:49:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Py4JJavaError: An error occurred while calling o4765.collectToPython.
: java.lang.NoSuchMethodError: 'org.apache.spark.internal.config.ConfigEntry org.apache.spark.sql.internal.SQLConf$.ENABLE_TWOLEVEL_AGG_MAP_PARTIAL_ONLY()'
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.checkIfFastHashMapSupported(HashAggregateExec.scala:674)
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.enableTwoLevelHashMap(HashAggregateExec.scala:681)
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.doProduceWithKeys(HashAggregateExec.scala:698)
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.doProduce(HashAggregateExec.scala:151)
	at org.apache.spark.sql.execution.CodegenSupport.$anonfun$produce$1(WholeStageCodegenExec.scala:96)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:222)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQu

23/01/24 21:49:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:49:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:49:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:49:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/24 21:49:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Py4JJavaError: An error occurred while calling o4815.collectToPython.
: java.lang.NoSuchMethodError: 'org.apache.spark.internal.config.ConfigEntry org.apache.spark.sql.internal.SQLConf$.ENABLE_TWOLEVEL_AGG_MAP_PARTIAL_ONLY()'
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.checkIfFastHashMapSupported(HashAggregateExec.scala:674)
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.enableTwoLevelHashMap(HashAggregateExec.scala:681)
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.doProduceWithKeys(HashAggregateExec.scala:698)
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.doProduce(HashAggregateExec.scala:151)
	at org.apache.spark.sql.execution.CodegenSupport.$anonfun$produce$1(WholeStageCodegenExec.scala:96)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:222)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQu